# Ejercicio 7: Bases de Datos Vectoriales

## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [2]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

Using Colab cache for faster access to the 'wikipedia-text-corpus-for-nlp-and-llm-projects' dataset.


,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [4]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

(   doc_id  chunk_id                                               text
 0       0         0  Anovo Anovo (formerly A Novo) is a computer se...
 1       1         0  Battery indicator A battery indicator (also kn...
 2       1         1  ad battery when in reality it indicates a prob...
 3       1         2  s that an internal standby battery needs repla...
 4       1         3  increase; in many cases the EMF remains more o...,
 79104)

In [5]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [10]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
from google.colab import drive
drive.mount('/content/drive')

embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

np.save(
    "/content/drive/MyDrive/embeddings.npy",
    embeddings
)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyboardInterrupt: 

In [6]:
embeddings = np.load(
    "/content/drive/MyDrive/embeddings.npy"
)

In [7]:
print(embeddings.shape, embeddings.dtype)

(79104, 768) float32


In [8]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

In [ ]:
!pip install faiss-cpu

In [9]:
# código base para FAISS
import faiss
import numpy as np

# Asumiendo `embeddings` en un array NxD
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

D, I = index.search(query_vec, k=10)

---

In [ ]:
!pip install qdrant-client pymilvus weaviate-client chromadb psycopg2-binary pgvector

## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`

### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`

### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?
- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?


In [15]:
# --- Parte 3: Qdrant ---
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# 1. Levantar instancia (En memoria para este ejercicio)
client_qdrant = QdrantClient(":memory:")

# 2. Crear colección
# La dimensión de e5-base-v2 es 768. Usamos Cosine porque los embeddings están normalizados.
collection_name = "wiki_chunks"
client_qdrant.recreate_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

# 3. Insertar datos
points = []
for idx, row in chunks_df.iterrows():
    points.append(PointStruct(
        id=int(idx),  # Qdrant prefiere enteros o UUIDs
        vector=embeddings[idx].tolist(),
        payload={"text": row["text"], "doc_id": row["doc_id"]} # Metadata
    ))

# Insertar por lotes
client_qdrant.upload_points(
    collection_name=collection_name,
    points=points
)
print("Datos indexados en Qdrant.")

# 4. Función de búsqueda
def qdrant_search(query_embedding, k=5):
    search_result = client_qdrant.query_points(
        collection_name=collection_name,
        query=query_embedding.flatten().tolist(),
        limit=k
    )

    results = []
    for hit in search_result.points:
        results.append((hit.id, hit.score, hit.payload["text"], hit.payload))
    return results

/tmp/ipython-input-2865029923.py:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client_qdrant.recreate_collection(


Datos indexados en Qdrant.


/usr/local/lib/python3.12/dist-packages/qdrant_client/qdrant_client.py:1868: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 79104 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  return self._client.upload_points(


In [25]:
# Ejecutar ejemplo
q_results = qdrant_search(query_vec, k=5)
for res in q_results:
    print(f"ID: {res[0]}\n\t Score: {res[1]:.4f} | Text: {res[2][:50]}...")

ID: 10176
	 Score: 0.8703 | Text: Battery tester A battery tester is an electronic d...
ID: 1
	 Score: 0.8618 | Text: Battery indicator A battery indicator (also known ...
ID: 10177
	 Score: 0.8401 | Text: ing procedure, according to the type of battery be...
ID: 37406
	 Score: 0.8391 | Text: ils. One was connected via a series resistor to th...
ID: 71872
	 Score: 0.8386 | Text: is achieved. Accepted average float voltages for l...


## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.
2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)
3. Insertar `N` embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).

### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?


In [26]:
# --- Parte 4: Milvus ---
from pymilvus import MilvusClient

# 1. Conectar
client_milvus = MilvusClient("milvus_demo.db")

# 2. Crear colección
if client_milvus.has_collection("wiki_collection"):
    client_milvus.drop_collection("wiki_collection")

client_milvus.create_collection(
    collection_name="wiki_collection",
    dimension=768,
    metric_type="COSINE" # O "IP" (Inner Product) si están normalizados
)

# 3. Insertar datos
data_milvus = []
for idx, row in chunks_df.iterrows():
    data_milvus.append({
        "id": int(idx),
        "vector": embeddings[idx].tolist(),
        "text": row["text"],
        "doc_id": int(row["doc_id"])
    })

def batch_insert_milvus(client, collection_name, data, batch_size=500):
    total = len(data)
    for i in range(0, total, batch_size):
        batch = data[i:i + batch_size]
        client.insert(collection_name=collection_name, data=batch)

batch_insert_milvus(
    client=client_milvus,
    collection_name="wiki_collection",
    data=data_milvus,
    batch_size=500
)
print("Datos indexados en Milvus.")

# 4. Función de búsqueda
def milvus_search(query_embedding, k=5):
    # Milvus por defecto usa índices ANN (HNSW generalmente en Lite)
    search_res = client_milvus.search(
        collection_name="wiki_collection",
        data=[query_embedding.flatten().tolist()],
        limit=k,
        output_fields=["text", "doc_id"]
    )

    results = []
    for hit in search_res[0]:
        results.append((hit["id"], hit["distance"], hit["entity"]["text"], {"doc_id": hit["entity"]["doc_id"]}))
    return results

Datos indexados en Milvus.


In [29]:
# Mini experimento
print("--- K=5 ---")
m_results_5 = milvus_search(query_vec, k=5)
for res in m_results_5:
    print(f"ID: {res[0]}\n\t Score: {res[1]:.4f} | Text: {res[2][:50]}...")

--- K=5 ---
ID: 10176
	 Score: 0.8703 | Text: Battery tester A battery tester is an electronic d...
ID: 1
	 Score: 0.8618 | Text: Battery indicator A battery indicator (also known ...
ID: 10177
	 Score: 0.8401 | Text: ing procedure, according to the type of battery be...
ID: 37406
	 Score: 0.8391 | Text: ils. One was connected via a series resistor to th...
ID: 71872
	 Score: 0.8386 | Text: is achieved. Accepted average float voltages for l...


## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)
3. Insertar objetos con:
   - propiedades + vector
4. Consultar por similitud (Top-k) con `query_embedding`.
5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata

### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
- ¿Cómo describirías el trade-off de complejidad vs expresividad?


In [30]:
# --- Parte 5: Weaviate ---
import weaviate
from weaviate.classes.config import Configure, Property, DataType

# 1. Conectar (Embedded corre Weaviate dentro del proceso Python)
# NOTA: Si esto falla en tu entorno, requiere un docker corriendo en localhost:8080
try:
    client_weaviate = weaviate.connect_to_embedded()
except:
    # Fallback si tienes docker corriendo
    client_weaviate = weaviate.connect_to_local()

# 2. Definir Esquema
collection_name = "WikiChunk"

# Borrar si existe para limpieza
try:
    client_weaviate.collections.delete(collection_name)
except:
    pass

# Crear colección con vectorizer configurado a 'none' porque traemos nuestros propios embeddings
client_weaviate.collections.create(
    name=collection_name,
    properties=[
        Property(name="text", data_type=DataType.TEXT),
        Property(name="doc_id_meta", data_type=DataType.INT), # 'doc_id' es reservado a veces
    ],
    vectorizer_config=Configure.Vectorizer.none()
)

# 3. Insertar objetos
chunks_collection = client_weaviate.collections.get(collection_name)

# Usamos batch para velocidad
with chunks_collection.batch.dynamic() as batch:
    for idx, row in chunks_df.iterrows():
        batch.add_object(
            properties={
                "text": row["text"],
                "doc_id_meta": row["doc_id"]
            },
            vector=embeddings[idx].tolist()
        )

print("Datos indexados en Weaviate.")

# 4. Consultar
def weaviate_search(query_embedding, k=5):
    chunks = client_weaviate.collections.get(collection_name)
    response = chunks.query.near_vector(
        near_vector=query_embedding.flatten().tolist(),
        limit=k,
        return_metadata=["distance"]
    )

    results = []
    for o in response.objects:
        # Weaviate devuelve distancia, score = 1 - distance (aprox)
        results.append((o.uuid, o.metadata.distance, o.properties["text"], o.properties))
    return results

INFO:weaviate-client:Started /root/.cache/weaviate-embedded: process ID 49839


Datos indexados en Weaviate.


In [33]:
w_results

[(_WeaviateUUIDInt('7e1bb27f-79b1-4085-8aa9-914c96dcf1b5'),
  0.12965166568756104,
  "Battery tester A battery tester is an electronic device intended for testing the state of an electric battery, going from a simple device for testing the charge actually present in the cells and/or its voltage output, to a more comprehensive testing of the battery's condition, namely its capacity for accumulating charge and any possible flaws affecting the battery's performance and security. The most simple battery tester is a DC ammeter, that indicates the battery's charge rate. DC voltmeters can be used to estimate the charge rate of a battery, provided that its nominal voltage is known. There are many types of integrated battery testers, each one corresponding to a specific condition testing procedure, according to the type of battery being tested, such as the â€œ421â€\x9d test for lead-ac",
  {'text': "Battery tester A battery tester is an electronic device intended for testing the state of an ele

In [38]:
w_results = weaviate_search(query_vec, k=5)
for res in w_results:
    print(f"ID: {res[3]["doc_id_meta"]}\n\t Dist: {res[1]:.4f} | Text: {res[2][:50]}...")

ID: 1391
	 Dist: 0.1297 | Text: Battery tester A battery tester is an electronic d...
ID: 1
	 Dist: 0.1382 | Text: Battery indicator A battery indicator (also known ...
ID: 1391
	 Dist: 0.1599 | Text: ing procedure, according to the type of battery be...
ID: 5067
	 Dist: 0.1609 | Text: ils. One was connected via a series resistor to th...
ID: 9888
	 Dist: 0.1614 | Text: is achieved. Accepted average float voltages for l...


In [39]:
# Cerrar cliente al terminar
client_weaviate.close()

## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.
2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)
3. Consultar Top-k con `query_embedding`.

### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.

### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
- ¿Qué limitaciones ves para un sistema en producción?


In [40]:
# --- Parte 6: Chroma ---
import chromadb

# 1. Crear cliente (Efímero o persistente)
chroma_client = chromadb.Client()

# 2. Crear colección
# cosine es default en chroma para embeddings
collection_chroma = chroma_client.get_or_create_collection(
    name="wiki_test"
)

# 3. Insertar
# Chroma maneja lotes automáticamente si le pasas listas
ids = [str(i) for i in chunks_df.index] # IDs deben ser strings
docs = chunks_df["text"].tolist()
metadatas = [{"doc_id": int(d)} for d in chunks_df["doc_id"]]
embs = embeddings.tolist()

def batch_insert_chroma(collection, ids, docs, embs, metadatas, batch_size=1000):
    total = len(ids)
    for i in range(0, total, batch_size):
        collection.add(
            ids=ids[i:i + batch_size],
            documents=docs[i:i + batch_size],
            embeddings=embs[i:i + batch_size],
            metadatas=metadatas[i:i + batch_size],
        )

batch_insert_chroma(
    collection=collection_chroma,
    ids=ids,
    docs=docs,
    embs=embs,
    metadatas=metadatas,
    batch_size=1000
)

print("Datos indexados en Chroma.")

# 4. Buscar
def chroma_search(query_embedding, k=5):
    results = collection_chroma.query(
        query_embeddings=[query_embedding.flatten().tolist()],
        n_results=k
    )

    # Chroma devuelve listas de listas (una por cada query)
    output = []
    for i in range(len(results["ids"][0])):
        output.append((
            results["ids"][0][i],
            results["distances"][0][i],
            results["documents"][0][i],
            results["metadatas"][0][i]
        ))
    return output

Datos indexados en Chroma.


In [43]:
c_results = chroma_search(query_vec, k=5)
for res in c_results:
    print(f"ID: {res[3]['doc_id']}\n\t Score: {res[1]:.4f} | Text: {res[2][:50]}...")

ID: 1391
	 Score: 0.2593 | Text: Battery tester A battery tester is an electronic d...
ID: 1
	 Score: 0.2764 | Text: Battery indicator A battery indicator (also known ...
ID: 1391
	 Score: 0.3198 | Text: ing procedure, according to the type of battery be...
ID: 5067
	 Score: 0.3217 | Text: ils. One was connected via a series resistor to th...
ID: 9888
	 Score: 0.3228 | Text: is achieved. Accepted average float voltages for l...


## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.
2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.

### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?
- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?


In [ ]:
!pip install duckdb

In [10]:
# --- Parte 7: SQL + Vectores (DuckDB) ---
import duckdb

# 1. Conexión
con = duckdb.connect(database=':memory:')

# 2. Crear tabla
con.execute("""
CREATE TABLE wiki_docs (
    id INTEGER,
    content VARCHAR,
    embedding FLOAT[]
);
""")

# 3. Insertar datos
print("Insertando datos en DuckDB...")
data_to_insert = []
for idx, row in chunks_df.iterrows():
    data_to_insert.append(
        (int(idx), row["text"], embeddings[idx].tolist())
    )

con.executemany(
    "INSERT INTO wiki_docs VALUES (?, ?, ?)",
    data_to_insert
)
print(f"Total insertado: {len(data_to_insert)} filas.")

# 4. Función de búsqueda tipo pgvector
def pgvector_search(query_embedding, k=5):
    query_list = query_embedding.flatten().tolist()

    sql = """
    WITH query_vec AS (
        SELECT ?::FLOAT[] AS q
    )
    SELECT
        w.id,
        (
            -- dot product
            list_sum(
                list_transform(
                    list_zip(w.embedding, q),
                    x -> x[1] * x[2]
                )
            )
            /
            (
                sqrt(
                    list_sum(
                        list_transform(w.embedding, x -> x * x)
                    )
                )
                *
                sqrt(
                    list_sum(
                        list_transform(q, x -> x * x)
                    )
                )
            )
        ) AS score,
        w.content
    FROM wiki_docs w, query_vec
    ORDER BY score DESC
    LIMIT ?;
    """

    rows = con.execute(sql, [query_list, k]).fetchall()

    results = []
    for r in rows:
        results.append((r[0], r[1], r[2], {}))

    return results

Insertando datos en DuckDB...
Total insertado: 79104 filas.


In [11]:
sql_results = pgvector_search(query_vec, k=5)
for res in sql_results:
    print(f"ID: {res[0]}\n\t Score: {res[1]:.4f} | Text: {res[2][:50]}...")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

ID: 10176
	 Score: 0.8703 | Text: Battery tester A battery tester is an electronic d...
ID: 1
	 Score: 0.8618 | Text: Battery indicator A battery indicator (also known ...
ID: 10177
	 Score: 0.8401 | Text: ing procedure, according to the type of battery be...
ID: 37406
	 Score: 0.8391 | Text: ils. One was connected via a series resistor to th...
ID: 71872
	 Score: 0.8386 | Text: is achieved. Accepted average float voltages for l...
